In [1]:
import requests
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import datetime

load_dotenv(find_dotenv())

client = OpenAI()

def get_opening_hours():
    try:
        response = requests.get('http://localhost:8000/opening-hours')
        if response.status_code == 200:
            return response.json()  # Assuming the API returns JSON data
        else:
            return "Error: Unable to fetch opening hours."
    except requests.exceptions.RequestException as e:
        return f"Request failed: {e}"

def how_late():
    return datetime.datetime.now().strftime("%H:%M:%S")  # Returns current time in HH:MM:SS format

file = client.files.create(
    file=open("knowledge.txt", "rb"),
    purpose='assistants'
)

In [2]:
assistant = client.beta.assistants.create(
    name="Summariser",
    description="You are great at assisting customers of a bank.",
    model="gpt-4-turbo-preview",
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {
            "type": "function",
            "function": {
                "name": "get_opening_hours",
                "description": "Get the opening hours of the bank",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "how_late",
                "description": "Get the current time",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        }
    ],
    file_ids=[file.id]
)
print(assistant)

Assistant(id='asst_ePaIeDp60QcWt20dkKAfI2k4', created_at=1706880983, description='You are great at assisting customers of a bank.', file_ids=['file-sh7beN6SYVBZTRvw6iVMt11x'], instructions=None, metadata={}, model='gpt-4-turbo-preview', name='Summariser', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter'), ToolRetrieval(type='retrieval'), ToolFunction(function=FunctionDefinition(name='get_opening_hours', description='Get the opening hours of the bank', parameters={'type': 'object', 'properties': {}, 'required': []}), type='function'), ToolFunction(function=FunctionDefinition(name='how_late', description='Get the current time', parameters={'type': 'object', 'properties': {}, 'required': []}), type='function')])


In [3]:
thread = client.beta.threads.create()

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Wie viel kostet ein Darlehen pro Jahr?"
)

In [5]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  tools=[{"type": "code_interpreter"}, {"type": "retrieval"}] # allow to override assistant tools
)

In [6]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(run.status)

in_progress


Polling is required the get the current status

In [7]:
import time

def get_run_status(client, thread_id, run_id):
    try:
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        return {"status": run.status, "run": run}
    except Exception as e:
        print(f"Error retrieving run status: {e}")
        return {"status": None, "run": None}

def poll_run_status(client, thread_id, run_id):
    while True:
        result = get_run_status(client, thread_id, run_id)
        status = result["status"]
        run = result["run"]
        if status in ['completed', 'cancelled', 'failed', 'requires_action']:
            print(f"Run has finished or requires action with status: {status}")
            break
        else:
            print("Run is not finished yet...")
            time.sleep(2)

    return {"status": status, "run": run}

In [8]:
result = poll_run_status(client=client, thread_id=thread.id, run_id=run.id)

Run has finished or requires action with status: completed


In [9]:
print(result["run"])

Run(id='run_vmD0dWebLjYjQb9z5gc7RgbR', assistant_id='asst_ePaIeDp60QcWt20dkKAfI2k4', cancelled_at=None, completed_at=1706881014, created_at=1706881000, expires_at=None, failed_at=None, file_ids=['file-sh7beN6SYVBZTRvw6iVMt11x'], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt-4-turbo-preview', object='thread.run', required_action=None, started_at=1706881000, status='completed', thread_id='thread_yp7ZhDA1FxWO5GBHf5TU3eXB', tools=[ToolAssistantToolsCode(type='code_interpreter'), ToolAssistantToolsRetrieval(type='retrieval')], usage=Usage(completion_tokens=129, prompt_tokens=1455, total_tokens=1584))


In [10]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print(messages.data)

[ThreadMessage(id='msg_t8FGw4dM7ht3Pey5WhU1TUpL', assistant_id='asst_ePaIeDp60QcWt20dkKAfI2k4', content=[MessageContentText(text=Text(annotations=[], value='Der effektive Jahreszins (APR) für persönliche Darlehen beginnt ab 4,5%. Bei Hypothekenoptionen variieren die Zinssätze je nach Produkt, wobei eine 30-jährige Festhypothek ab 3,75% verfügbar ist. Weitere detaillierte Angaben zu den Kosten spezifischer Darlehensformen sind in den bereitgestellten Informationen nicht enthalten.'), type='text')], created_at=1706881005, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_vmD0dWebLjYjQb9z5gc7RgbR', thread_id='thread_yp7ZhDA1FxWO5GBHf5TU3eXB'), ThreadMessage(id='msg_4KPqiJypljm2hDC2e25mU8Aw', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Wie viel kostet ein Darlehen pro Jahr?'), type='text')], created_at=1706880995, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_yp7ZhDA1FxWO5GBH

In [11]:
for message in messages.data:
    if message.content and message.content[0].text:
        print(f"Message: {message.content[0].text.value}")

Message: Der effektive Jahreszins (APR) für persönliche Darlehen beginnt ab 4,5%. Bei Hypothekenoptionen variieren die Zinssätze je nach Produkt, wobei eine 30-jährige Festhypothek ab 3,75% verfügbar ist. Weitere detaillierte Angaben zu den Kosten spezifischer Darlehensformen sind in den bereitgestellten Informationen nicht enthalten.
Message: Wie viel kostet ein Darlehen pro Jahr?


### Function usage

In [12]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Wie spät ist es?"
)

In [13]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
)

In [14]:
run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

In [16]:
result = poll_run_status(client=client, thread_id=thread.id, run_id=run.id)

Run has finished or requires action with status: requires_action


In [17]:
run.required_action.submit_tool_outputs.tool_calls

[RequiredActionFunctionToolCall(id='call_UNgmQb0xFlfi9Ku5ZHvrxsN4', function=Function(arguments='{}', name='how_late'), type='function')]

In [21]:
for tool_call in run.required_action.submit_tool_outputs.tool_calls:
    tool_call_id = tool_call.id
    function_name = tool_call.function.name
    print(f"Executing function: {function_name}")

    if function_name == "get_opening_hours":
        output = get_opening_hours()
    elif function_name == "how_late":
        output = how_late()
    else:
        output = "Function not recognized"
        print("Error: Function not recognized.")
    print(f"Output from function call:  {output}")
        
    client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {"tool_call_id": tool_call_id, "output": output},
    ])

Executing function: how_late
Output from function call:  14:47:04


In [23]:
print(output)

14:47:04


In [29]:
for message in messages.data:
    if message.content and message.content[0].text:
        print(f"Message: {message.content[0].text.value}")

Message: Es ist jetzt 14:47 Uhr.
Message: Wie spät ist es?
Message: Der effektive Jahreszins (APR) für persönliche Darlehen beginnt ab 4,5%. Bei Hypothekenoptionen variieren die Zinssätze je nach Produkt, wobei eine 30-jährige Festhypothek ab 3,75% verfügbar ist. Weitere detaillierte Angaben zu den Kosten spezifischer Darlehensformen sind in den bereitgestellten Informationen nicht enthalten.
Message: Wie viel kostet ein Darlehen pro Jahr?


### Delete everthing after using it!

In [ ]:
# deleted_assistant_file = client.beta.assistants.files.delete(
#     assistant_id=assistant.id,
#     file_id=file.id
# )
# print(deleted_assistant_file)

# response = client.beta.assistants.delete(assistant.id)
# print(response)